<font size="45">Microsoft Movie Studios Analysis</font>

<font size="5">Overview</font>

This project analyses which directors and genres Microsoft Movie Studios should consider when producing movies based on the ratings each recieved. Descriptive analysis of (IMDb) movie data shows that some directors have higher ratings on their movies compared to others and certain genres also have higher ratings. Microsoft movie studios can use these findings to hire outstandings directors to produce movies that everyone will enjoy.

<font size="5">Business Understanding</font>

Microsoft Movie Studios is looking to start showing orignal movies to beat out all competitors by being the most highly used streaming service. They are willing to do whatever it takes with no limitations in order to acomplish this goal. Using previous(IMDb) movie data that contains information on genres, directors, average ratings, and number of votes has allowed us to determine a guaranteed path to sucess for Microsoft Movie Studios and what aspects to consider.

<font size="5">Data Understandng</font>

In [2]:
import pandas as pd
import numpy as np
import seaborn. as sns
import matplotlib as plt

<font size="5">Data Preparation </font>

### Data Cleaning

### Merging Datasets

<font size="5">Data Visualization</font>

<font size="5">Conclusion</font>

This Data analysis leads to three recommendations that will allow Microsoft Movie Studios to come out as being the most popular streaming service in business.
- Look to hire directors who have made at least 10 productions that have high ratings. Directors who are dominating in the productions industry are {LIST THE MOST POPULAR DIRECTORS}
- Focus on streaming orignal movies that are within the most popular genres. Focal point genres that are enjoyed the most are {LIST MOST POPULAR GENRES}
- 


<font size="5">Next Step</font>

In [ ]:
Addition analysis 